In [1]:
# Imports of needed libraries
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash_bootstrap_templates import load_figure_template

In [59]:
# Df All Athletes
athlete_events = pd.read_csv("../data/final_df.csv")

# Df Swedish Athletes
sweden_athletes = pd.DataFrame(athlete_events[athlete_events["NOC"] == "SWE"])

gender_ratios = pd.read_csv("../data/gender_ratios.csv")

C:\Users\yonis\AppData\Local\Temp\ipykernel_6352\1946786575.py:2: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
participants_medals= athlete_events.groupby(['Year','Country', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})

participants_medals_start = participants_medals.groupby(['Country','Continent' , 
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

participants_medals_sport = athlete_events.groupby(['Year', 'Country', 'Sport', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})


In [32]:
participants_medals_sport = athlete_events.groupby(['Year','Country', 'Sport', 'Continent','Country_latitude', 'Country_longitude','Continent_latitude', 'Continent_longitude'], as_index=False)[['Name', 'Medal']].agg(
    {'Name': 'nunique', 'Medal': 'count'})

participants_medals_sport_total = participants_medals_sport.groupby(['Country','Sport' , 'Continent',
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

participants_medals_sport_year = participants_medals_sport.groupby(['Year','Country','Sport' , 'Continent',
                                                        'Country_latitude', 'Country_longitude','Continent_latitude', 
                                                        'Continent_longitude'], as_index=False)[['Name', 'Medal']].sum()

medal_distribution = athlete_events.groupby(['Year','Country', 'Sport', 'Medal']).size().reset_index(name="Number of Medals")



In [68]:
# Loading template for graphs
load_figure_template("quartz")

# This initializes a Dash web application, 
# applies the "QUARTZ" Bootstrap theme for styling, 
# and includes a viewport meta tag to ensure proper rendering on various devices.
app = Dash(__name__, 
           external_stylesheets=[dbc.themes.QUARTZ],
               meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

# Needs to be included for deploying on render
server = app.server

# Layout for App
app.layout = html.Div([
    dbc.Row(
        [
            dbc.Col(html.H2("Olympic Games Achievements 1896-2016", className="text-center text-primary")),
        ],
        className="mb-3 mt-3", # Adding marginal bottom and top
    ),
    dbc.Row([
            dcc.Dropdown(id='year_dropdown', className='text-info',
                        multi=True, 
                        options=[years for years in sorted(athlete_events['Year'].unique())], 
                        placeholder='Select Year',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'},
            ),
            dcc.Dropdown(id='sport_dropdown', className='text-info',
                        multi=True, 
                        options=[sports for sports in sorted(athlete_events['Sport'].unique())], 
                        placeholder='Select Sport',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'},
            ),
            dcc.Dropdown(id='season_dropdown', className='text-info',
                        multi=True, 
                        options=[season for season in sorted(athlete_events['Season'].unique())], 
                        placeholder='Select Season',
                        style={'width':'150px', 'margin-left': '10px', 'margin-right': '10px'})

    ], justify='center', class_name="mb-2 ml-50 mr-50"), 

    dbc.Row([
        dbc.Col([
                dcc.Dropdown(id='country_dropdown_left', 
                    className='ml-3 mr-3 mb-1 text-info', 
                    options=["Medals", "Gender ratio"], 
                    placeholder='Select to see participants with relation to medals or gender ratio'),
                    dcc.Graph(id="graph_1_left",    
                    figure={}),
                ], xs=12, sm=11, md=10, lg=5, width='auto'),
        dbc.Col([
                dcc.Dropdown(id='country_dropdown_right', 
                    className='ml-3 mr-3 mb-1 text-info', 
                    options=["Country", "Sport"], 
                    placeholder='Select to sort by country or sport'), 
                    dcc.Graph(id="graph_1_right",    
                    figure={})],
                    xs=12, sm=11, md=10, lg=5),
    ], justify='evenly'),


dbc.Row([
        dbc.Button("Reset", id='reset-button', className='float-left', n_clicks=0), # Callback to reset page
        dcc.Link("Contributors", href="https://github.com/DeerBay/OS-Project/graphs/contributors", target="_blank"), # "_blank": Opens the linked document in a new tab or window.
    ],style={"margin-top": "20px", "text-align": "center"})

])


@callback(
    Output("graph_1_left", "figure"),
    Input("year_dropdown", "value"),
    Input("sport_dropdown", "value"),
    Input("country_dropdown_left", "value")
)
def figure_one(years, sports, sort):
    if sort == "Medals" or sort in [None, "", []]:
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.scatter_mapbox(participants_medals_start, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760, hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1.2, opacity=0.5,
            title="Size according to count of participants")
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = participants_medals.query("Year==@years")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
            center=dict(lat=0, lon=0), zoom=1,
            title="Size according to count of participants")
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_sport.query("Sport==@sports")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800,width= 760, hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants")
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = participants_medals_sport_year.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.scatter_mapbox(df, lat="Country_latitude", 
            lon="Country_longitude", size="Name", color="Medal", 
            height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map",
            center=dict(lat=0, lon=0), zoom=1, 
            title="Size according to count of participants")
    elif sort =="Gender ratio":
        df = gender_ratios
        fig = px.scatter_mapbox(df, animation_frame = "Year", lat="Country_latitude",
                lon="Country_longitude", size="Count", color="Ratio", 
                height=800, width= 760,hover_name="Country",  mapbox_style="open-street-map", 
                center=dict(lat=0, lon=0), zoom=1, title="Gender ratios over the years")

        
    fig.update_layout(                       
    mapbox_style="white-bg",
    mapbox_layers=[
    {
    "below": 'traces',
    "sourcetype": "raster",
    "sourceattribution": "United States Geological Survey",
    "source": [
        "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
    ]
    }
    ])

    fig.update_mapboxes(bounds_east=180, bounds_west=-180, bounds_north=90, bounds_south=-90)
    

    return fig



@callback(
    Output("graph_1_right", "figure"),
    Input("year_dropdown", "value"),
    Input("sport_dropdown", "value"),
    Input("country_dropdown_right", "value"),
    Input("scountry_dropdown", "value")
)
def figure_two(years, sports, sort, country):
    if sort == "Sport" or sort in [None, "", []]:
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.sunburst(medal_distribution, values='Number of Medals', path=['Sport', 'Country'], height=800)
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Country'], height=800)
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Country'], height=800)
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Sport', 'Country'], height=800)
        return fig
    elif sort =="Country":   
        if (years in [None, "", []]) and (sports in [None, "", []]):
            fig = px.sunburst(medal_distribution, values='Number of Medals', path=['Country', 'Sport'], height=800)
        elif years not in [None, "", []] and sports in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            fig = px.sunburst(df, values='Number of Medals', path=['Country', 'Sport'], height=800)
        elif years in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Country', 'Sport'], height=800)
        elif years not in [None, "", []] and sports not in [None, "", []]:
            df = medal_distribution.query("Year==@years")
            df = df.query("Sport==@sports")
            fig = px.sunburst(df, values='Number of Medals', path=['Country', 'Sport'], height=800)
        return fig
        

if __name__ == "__main__":
    app.run(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


In [28]:
medal_distribution = athlete_events.groupby(['Year','Country', 'Sport', 'Medal']).size().reset_index(name="Number of Medals")

In [27]:
medal_distribution.query("Year==2016")

,Year,Country,Sport,Medal,Number of Medals
10352,2016,Algeria,Athletics,Silver,2
10353,2016,Argentina,Hockey,Gold,18
10354,2016,Argentina,Judo,Gold,1
10355,2016,Argentina,Sailing,Gold,2
10356,2016,Argentina,Tennis,Silver,1
...,...,...,...,...,...
10955,2016,Uzbekistan,Weightlifting,Gold,1
10956,2016,Uzbekistan,Wrestling,Bronze,3
10957,2016,Venezuela,Athletics,Silver,1
10958,2016,Venezuela,Boxing,Bronze,1


In [44]:
px.sunburst(medal_distribution, values='Number of Medals', path=['Sport', 'Country'], 
            #Remove background color
            template='plotly_white')
            # )

In [49]:
fig = px.scatter_mapbox(participants_medals_start, lat="Country_latitude", 
        lon="Country_longitude", size="Name", color="Medal", 
        height=800, width=700, hover_name="Country",  mapbox_style="open-street-map", 
        center=dict(lat=0, lon=0), zoom=1, 
        #remove purple background color
        template= "quartz", opacity=1,

        title="Size according to count of participants=")

# set bounds to only see one world map and not be able to scroll through repeated maps
fig.update_mapboxes(bounds_east=180, bounds_west=-180, bounds_north=90, bounds_south=-90)
fig.show()